Cell types in the mouse cortex and hippocampus revealed by single-cell RNA-seq

_Zeisel et al., Science 2015_

In [1]:
%matplotlib notebook
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

import multiprocessing
import warnings

In [2]:
from negbin_val_functions import *

In [3]:
warnings.simplefilter('ignore')
raw = pd.read_csv("/home/npapado/Documents/data/mousecort/expression_mRNA_17-Aug-2014.txt",
                   sep = "\t", header=None)
# apparently this includes only the 3005 cells used for clustering
# but the cells excluded were either low-quality or non-informative

In [4]:
metadata = raw[0:11]
counts = raw[11:]

read metadata

In [5]:
tissue = np.array(metadata.loc[0][2:], dtype=np.str_)
cell_group = np.array(metadata.loc[1][2:], dtype=np.int_)
total_mrna = np.array(metadata.loc[2][2:], dtype=np.int_)
well = np.array(metadata.loc[3][2:], dtype=np.int_)
sex = np.array(metadata.loc[4][2:], dtype=np.int_)
age = np.array(metadata.loc[5][2:], dtype=np.int_)
diameter = np.array(metadata.loc[6][2:], dtype=np.float)
cell_id = np.array(metadata.loc[7][2:], dtype=np.str_)
level1_class = np.array(metadata.loc[8][2:], dtype=np.str_)
level2_class = np.array(metadata.loc[9][2:], dtype=np.str_)

format data correctly

In [6]:
gene_names = np.array(counts[0])
gene_group = np.array(counts[1])
X = np.array(counts.loc[:, 2:], dtype = np.int_)

In [7]:
# remove genes with less than a combined 25 mRNA molecules
print(X.shape)
combmRNA = np.sum(X, axis=1)
cellmRNA = np.sum(X, axis=0)
print(combmRNA.shape)
keep = [i for i, s in enumerate(combmRNA) if s >= 25]
print(len(keep))
X = X[keep]
X = X.T
print(X.shape)

(19972, 3005)
(19972,)
15251
(3005, 15251)


isolate the subgroups and perform analysis for all of them independently

In [8]:
def run_subgroup(i):
    group = groups[i]
    print(i , "/", len(groups)-1, ",", group)
    keep = np.where(level2_class == group)[0]
    combmRNA = np.sum(X[keep,:], axis=0)
    genes = (combmRNA>0)
    G = sum(genes)

    norm_pvals, mu_res, s2_res = fit_normals(G, X[keep][:, genes])
    pois_pvals, lambda_res, pois_success = fit_poissons(G, X[keep][:, genes])
    negbin_pvals, a_res, b_res, nb_success = fit_negbins(G, X[keep][:, genes])

    norm_all = [norm_pvals, mu_res, s2_res]
    pois_all = [pois_pvals, lambda_res, pois_success]
    negbin_all = [negbin_pvals, a_res, b_res, nb_success]

    save_all("zeisel", group, norm_all, pois_all, negbin_all)

In [ ]:
groups = np.unique(level2_class)
groups = np.delete(groups, np.where(groups == "(none)"))

In [ ]:
warnings.simplefilter('ignore')
pool = multiprocessing.Pool(20)
maps = pool.map(run_subgroup, range(0, len(groups)))

2 / 46 , CA1Pyr1
1 / 46 , Astro2
3 / 46 , CA1Pyr2
0 / 46 , Astro1
4 / 46 , CA1PyrInt
5 / 46 , CA2Pyr2
6 / 46 , Choroid
8 / 46 , Epend
16 / 46 , Int16
7 / 46 , ClauPyr
9 / 46 , Int1
14 / 46 , Int14
15 / 46 , Int15
17 / 46 , Int2
18 / 46 , Int3
19 / 46 , Int4
10 / 46 , Int10
13 / 46 , Int13
11 / 46 , Int11
12 / 46 , Int12
20 / 46 , Int5
21 / 46 , Int6
22 / 46 , Int7
23 / 46 , Int8
24 / 46 , Int9
25 / 46 , Mgl1
26 / 46 , Mgl2
27 / 46 , Oligo1
28 / 46 , Oligo2
29 / 46 , Oligo3
30 / 46 , Oligo4
31 / 46 , Oligo5
32 / 46 , Oligo6
33 / 46 , Peric
34 / 46 , Pvm1
35 / 46 , Pvm2
36 / 46 , S1PyrDL
37 / 46 , S1PyrL23
38 / 46 , S1PyrL4
39 / 46 , S1PyrL5
40 / 46 , S1PyrL5a
41 / 46 , S1PyrL6


In [ ]:
from scipy.stats import norm
%matplotlib inline

for i in range(len(groups)):
    fig, ax = plt.subplots()
    group = groups[i]
    keep = np.where(level2_class == group)[0]
    sum1 = np.sum(X[keep, :], axis=1)
    scalings = sum1 / np.mean(sum1)
    s, mu, std = lognorm.fit(scalings)
    
    ax.hist(scalings, bins=25, normed=True, alpha=0.6, color='g')
    np.savetxt("zeisel/" + group + "_scalings.txt", scalings)

    # Plot the PDF.
    x = np.linspace(-1, 3, 100)
    p = lognorm.pdf(x, s, mu, std)
    ax.plot(x, p, 'k', linewidth=2)
    plt.savefig("zeisel/" + group + "_scalings.png", bbox_inches='tight')
    plt.close()
    print(group, mu, std)

In [ ]:
for group in groups:
    keep = np.where(level2_class == group)[0]
    gene_means = np.mean(X[keep, :], axis=0)
    np.savetxt("zeisel/" + group + "_genes.txt", gene_means)